In [1]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up paths to your dataset
data_dir = 'D:\TY B.Tech\Computer Vision\Course Project\Dataset'
cyst_dir = os.path.join(data_dir, 'cyst')
tumor_dir = os.path.join(data_dir, 'tumor')

# Split the data into training, validation, and testing sets
cyst_images = os.listdir(cyst_dir)
tumor_images = os.listdir(tumor_dir)

train_cyst, test_cyst = train_test_split(cyst_images, test_size=0.2, random_state=42)
train_tumor, test_tumor = train_test_split(tumor_images, test_size=0.2, random_state=42)

# Create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',  # Assuming two classes: cyst and tumor
    subset='training'
)

validation_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\TY B.Tech\\Computer Vision\\Course Project\\Dataset\\cyst'

In [ ]:
#Build a CNN Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
#Train a Model

history = model.fit(
    train_generator,
    epochs=10,  # Adjust as needed
    validation_data=validation_generator
)


In [ ]:
#Evaluate a Model

test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test accuracy: {test_accuracy}')


In [ ]:
#Use the Model for Inference
# Assuming you have a test image as 'test_image.jpg'
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = 'path_to_test_image.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)
if prediction[0] > 0.5:
    print('Tumor')
else:
    print('Cyst')